<a href="https://colab.research.google.com/github/GoryachevDaniil/DS-ML_Intro_stepik.ipynb/blob/main/DS_ML_Intro_stepik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
students_performance = pd.read_csv('/content/gdrive/My Drive/DS_ML_Intro_stepik/StudentsPerformance.csv', sep=',')

In [4]:
students_performance

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [5]:
students_performance.describe()

,math score,reading score,writing score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [6]:
students_performance.dtypes

gender                         object
race/ethnicity                 object
parental level of education    object
lunch                          object
test preparation course        object
math score                      int64
reading score                   int64
writing score                   int64
dtype: object

In [7]:
students_performance.groupby('gender').aggregate({'writing score' : 'mean'})

,writing score
gender,
female,72.467181
male,63.311203


In [8]:
students_performance.shape

(1000, 8)

In [9]:
students_performance.size

8000

In [10]:
students_performance.iloc[0:5, 0:]

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [11]:
students_performance.iloc[[0, 4], [0, -1, -2, -3]]

,gender,writing score,reading score,math score
0,female,74,72,72
4,male,75,78,76


In [12]:
students_performance_with_name = students_performance.iloc[[0, 4 , 9]]

In [13]:
students_performance_with_name.index = ['Nastya', 'Vasya', 'Katya']

In [14]:
students_performance_with_name

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
Nastya,female,group B,bachelor's degree,standard,none,72,72,74
Vasya,male,group C,some college,standard,none,76,78,75
Katya,female,group B,high school,free/reduced,none,38,60,50


In [15]:
students_performance_with_name.loc[['Vasya','Katya'], ['gender', 'writing score']]

,gender,writing score
Vasya,male,75
Katya,female,50


In [16]:
students_performance_with_name.iloc[:, 0] # pd.Series

Nastya    female
Vasya       male
Katya     female
Name: gender, dtype: object

In [17]:
my_series_1 = pd.Series([1, 2, 3], index=['Nastya', 'Vasya', 'Katya'])
my_series_2 = pd.Series([4, 5, 6], index=['Nastya', 'Vasya', 'Katya'])

In [18]:
pd.DataFrame({'col_name_1' : my_series_1, 'col_name_2' : my_series_2})

,col_name_1,col_name_2
Nastya,1,4
Vasya,2,5
Katya,3,6


In [19]:
students_performance_with_name['gender'] # pd.Series

Nastya    female
Vasya       male
Katya     female
Name: gender, dtype: object

In [20]:
students_performance_with_name[['gender']] # pd.DataFrame

,gender
Nastya,female
Vasya,male
Katya,female


In [21]:
titanic = pd.read_csv('/content/gdrive/My Drive/DS_ML_Intro_stepik/titanic.csv', sep=',')

In [23]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
titanic.shape

(891, 12)

In [26]:
titanic.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [27]:
titanic.dtypes.value_counts()

int64      5
object     5
float64    2
dtype: int64

In [30]:
students_performance.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [37]:
mean_writing_score = students_performance['writing score'].mean()

In [42]:
query = (students_performance['writing score'] > mean_writing_score) & (students_performance.gender == 'female')

In [38]:
students_performance.loc[students_performance['writing score'] > mean_writing_score]

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
4,male,group C,some college,standard,none,76,78,75
5,female,group B,associate's degree,standard,none,71,83,78
...,...,...,...,...,...,...,...,...
992,female,group D,associate's degree,free/reduced,none,55,76,76
993,female,group D,bachelor's degree,free/reduced,none,62,72,74
995,female,group E,master's degree,standard,completed,88,99,95
998,female,group D,some college,standard,completed,68,78,77


In [43]:
students_performance.loc[query]

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
5,female,group B,associate's degree,standard,none,71,83,78
6,female,group B,some college,standard,completed,88,95,92
...,...,...,...,...,...,...,...,...
992,female,group D,associate's degree,free/reduced,none,55,76,76
993,female,group D,bachelor's degree,free/reduced,none,62,72,74
995,female,group E,master's degree,standard,completed,88,99,95
998,female,group D,some college,standard,completed,68,78,77


In [36]:
students_performance.loc[students_performance.gender == 'female']

,gender,writing score
0,female,74
1,female,88
2,female,93
5,female,78
6,female,92
...,...,...
993,female,74
995,female,95
997,female,65
998,female,77


In [45]:
students_performance.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [68]:
# У какой доли студентов из датасэта в колонке lunch указано free/reduced?
len(students_performance.loc[students_performance['lunch'] == 'free/reduced']) / len(students_performance)

0.355

In [70]:
# Как различается среднее и дисперсия оценок по предметам у групп студентов со стандартным или урезанным ланчем?
students_performance['lunch'].value_counts()

standard        645
free/reduced    355
Name: lunch, dtype: int64

In [87]:
standart_lunch_query = students_performance['lunch'] == 'standard'
free_reduced_lunch_query = students_performance['lunch'] == 'free/reduced'

In [88]:
std_lunch_students = students_performance.loc[standart_lunch_query, ['math score', 'reading score', 'writing score']]
f_r_lunch_students = students_performance.loc[free_reduced_lunch_query, ['math score', 'reading score', 'writing score']]

In [89]:
print('std_lunch_students:')
print(f"Mean:\nMath: {std_lunch_students['math score'].mean()}\nRading: {std_lunch_students['reading score'].mean()}\nWriting: {std_lunch_students['writing score'].mean()}\n")
print(f"Std:\nMath: {std_lunch_students['math score'].std()}\nRading: {std_lunch_students['reading score'].std()}\nWriting: {std_lunch_students['writing score'].std()}\n")
print('f_r_lunch_students:')
print(f"Mean:\nMath: {f_r_lunch_students['math score'].mean()}\nRading: {f_r_lunch_students['reading score'].mean()}\nWriting: {f_r_lunch_students['writing score'].mean()}\n")
print(f"Std:\nMath: {f_r_lunch_students['math score'].std()}\nRading: {f_r_lunch_students['reading score'].std()}\nWriting: {f_r_lunch_students['writing score'].std()}\n")

std_lunch_students:
Mean:
Math: 70.03410852713178
Rading: 71.65426356589147
Writing: 70.8232558139535

Std:
Math: 13.653500996457035
Rading: 13.830602308284629
Writing: 14.339486981713932

f_r_lunch_students:
Mean:
Math: 58.92112676056338
Rading: 64.65352112676057
Writing: 63.02253521126761

Std:
Math: 15.159956131640516
Rading: 14.895339495848573
Writing: 15.43382261658457



In [91]:
students_performance = students_performance.rename(columns = {
                   'parental level of education' : 'parental_level_of_education',
                   'test preparation course' : 'test_preparation_course',
                   'math score' : 'math_score',	
                   'reading score' : 'reading_score',
                   'writing score' : 'writing_score'})

In [95]:
students_performance.head()

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [101]:
students_performance.query('math_score > 72 & gender == "female"').head()

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
2,female,group B,master's degree,standard,none,90,95,93
6,female,group B,some college,standard,completed,88,95,92
36,female,group D,associate's degree,standard,none,74,81,83
38,female,group D,associate's degree,free/reduced,completed,75,90,88
56,female,group E,associate's degree,standard,completed,82,85,86


In [102]:
wr_sc_qr = 78
students_performance.query('writing_score > @wr_sc_qr').head()

,gender,race/ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
6,female,group B,some college,standard,completed,88,95,92
16,male,group C,high school,standard,none,88,89,86
24,male,group D,bachelor's degree,free/reduced,completed,74,71,80


In [105]:
score_cols = [i for i in list(students_performance) if 'score' in i] # не pandas way
score_cols

['math_score', 'reading_score', 'writing_score']

In [106]:
students_performance[score_cols].head() # не pandas way

,math_score,reading_score,writing_score
0,72,72,74
1,69,90,88
2,90,95,93
3,47,57,44
4,76,78,75


In [108]:
students_performance.filter(like='score') # pandas way

,math_score,reading_score,writing_score
0,72,72,74
1,69,90,88
2,90,95,93
3,47,57,44
4,76,78,75
...,...,...,...
995,88,99,95
996,62,55,55
997,59,71,65
998,68,78,77


In [112]:
students_performance_with_name.filter(like='s', axis=0) # pandas way

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
Nastya,female,group B,bachelor's degree,standard,none,72,72,74
Vasya,male,group C,some college,standard,none,76,78,75


In [ ]:
# Важной частью нашего курса будут задачи на программирование. Давайте попробуем решить первую задачу вместе.
# В переменной df  сохранен датафрэйм с произвольным числом колонок и строк. Отберите колонки, в которых есть '-' в датафрэйме df (вот соответствующий датасэт). Сохраните их в переменную selected_columns
# Если в условии задачи сообщается, что в некой переменной уже сохранен датафрэйм или другой объект, не нужно пересоздавать его самостоятельно, сразу начинайте работать с этой переменной.

In [114]:
df = pd.read_csv('/content/gdrive/My Drive/DS_ML_Intro_stepik/column_hell.csv', sep=',')
df

,yrh&6,ohku,q,hfxx,umeca-4,apj,g,ijvnixxo,kudbpyw-3,upubjv,...,fi,clsruyy,g&6,qo$6,ap,o,sladymzr,tif,wuikzxt,wzuvc
0,0.288397,0.911359,0.488069,0.074599,0.016124,0.090555,0.560810,0.110975,0.430787,0.548763,...,0.793122,0.070335,0.145115,0.686579,0.071681,0.563068,0.815201,0.081902,0.333237,0.185148
1,0.861276,0.134987,0.701885,0.897921,0.507115,0.740151,0.963179,0.712024,0.671662,0.497596,...,0.477181,0.767931,0.996055,0.948774,0.054327,0.087341,0.072166,0.856081,0.120481,0.622113
2,0.775577,0.832874,0.862665,0.831049,0.003274,0.703112,0.509795,0.573296,0.286411,0.282792,...,0.421542,0.770413,0.749468,0.395022,0.307752,0.535575,0.341393,0.971201,0.714558,0.702453
3,0.857910,0.555818,0.598305,0.473988,0.784139,0.427963,0.725103,0.286610,0.863717,0.545562,...,0.240252,0.010355,0.759912,0.965837,0.110719,0.439488,0.816526,0.111621,0.373365,0.461615
4,0.748652,0.451725,0.252619,0.584361,0.011265,0.051115,0.755444,0.627191,0.206172,0.955043,...,0.846165,0.043458,0.209347,0.028276,0.700446,0.203503,0.181903,0.428842,0.099398,0.175227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.665300,0.744322,0.377087,0.809735,0.220817,0.248306,0.131914,0.217087,0.434444,0.852261,...,0.469934,0.940416,0.012272,0.519183,0.231462,0.778445,0.979528,0.742654,0.070859,0.595118
96,0.344394,0.182979,0.590196,0.483025,0.474131,0.031048,0.672095,0.573249,0.176011,0.847096,...,0.954133,0.073519,0.708749,0.942929,0.050652,0.902627,0.478454,0.895962,0.254965,0.963576
97,0.694568,0.507591,0.328004,0.333993,0.091637,0.356788,0.541017,0.040395,0.871606,0.680075,...,0.300616,0.172525,0.984465,0.645257,0.184107,0.322045,0.011599,0.197995,0.841725,0.695915
98,0.692840,0.220333,0.729843,0.202430,0.360106,0.980271,0.913669,0.549889,0.172385,0.049770,...,0.188240,0.053409,0.928515,0.135282,0.352835,0.046042,0.667842,0.633934,0.547764,0.232205


In [116]:
selected_columns = df.filter(like='-')
selected_columns

,umeca-4,kudbpyw-3,g-3,v-9,fla-6,hwtbyrkp-5
0,0.016124,0.430787,0.202407,0.725508,0.305076,0.979296
1,0.507115,0.671662,0.864249,0.867818,0.358859,0.160520
2,0.003274,0.286411,0.928865,0.719159,0.340235,0.734612
3,0.784139,0.863717,0.175446,0.554744,0.049325,0.621132
4,0.011265,0.206172,0.044151,0.163643,0.048806,0.437836
...,...,...,...,...,...,...
95,0.220817,0.434444,0.601738,0.146333,0.918888,0.216499
96,0.474131,0.176011,0.359793,0.104707,0.257690,0.066934
97,0.091637,0.871606,0.121075,0.907699,0.313184,0.788592
98,0.360106,0.172385,0.015285,0.019571,0.060751,0.403166
